In [ ]:
import os
import json
import pandas as pd
import torchaudio
from datasets import Dataset
from transformers import Wav2Vec2Processor
import common_utils as utils

# 경로
wav_dir = "/kaggle/input/foregin-english/원천데이터/TS_2. 영어/1. 한국일반"
label_dir = "/kaggle/input/foregin-english/라벨링데이터/2. 영어/1. 한국일반"

audio_paths = []
transcripts = []

# 오디오-자막 매칭
for file in os.listdir(wav_dir):
    if file.endswith(".wav"):
        file_stem = os.path.splitext(file)[0]
        wav_path = os.path.join(wav_dir, file)
        transcript = utils.get_transcript_from_json(file_stem, label_dir)
        if transcript:
            audio_paths.append(wav_path)
            transcripts.append(transcript)

assert len(audio_paths) > 0, "오디오 데이터가 존재하지 않습니다."

# 데이터프레임 생성
df = pd.DataFrame({"path": audio_paths, "transcript": transcripts})

# 데이터 기본 통계 확인
print(f"데이터 개수: {len(df)}")
print("\n샘플 5개:")
print(df.head())

# 스크립트 길이 분포 확인
df['transcript_length'] = df['transcript'].apply(len)
print("\n스크립트 길이 통계:")
print(df['transcript_length'].describe())

# 데이터셋 생성
dataset = Dataset.from_pandas(df)

# 모델 및 프로세서 불러오기
model_name = "kresnik/wav2vec2-large-xlsr-korean"
processor = Wav2Vec2Processor.from_pretrained(model_name)

# 전처리 - wav -> speech
dataset = dataset.map(utils.preprocess, batched=False)

# 준비 - processor 적용
prepare_with_processor = partial(utils.prepare, processor=processor)
processed_dataset = dataset.map(prepare_with_processor, 
                               remove_columns=dataset.column_names, 
                               batched=False)

print(f"\n전처리 완료된 데이터셋 크기: {len(processed_dataset)}")
print(f"데이터셋 컬럼: {processed_dataset.column_names}")

# 전처리된 데이터셋 저장
processed_dataset.save_to_disk("processed_audio_dataset")